In [9]:
import cv2
import numpy as np
import time

In [10]:
net = cv2.dnn.readNet("path/to/yolov3.weights", "path/to/yolov3.cfg")

error: OpenCV(4.7.0) /io/opencv/modules/dnn/src/darknet/darknet_importer.cpp:210: error: (-212:Parsing error) Failed to open NetParameter file: path/to/yolov3.cfg in function 'readNetFromDarknet'


In [2]:
import cv2
import numpy as np
import time

# Load YOLO model
net = cv2.dnn.readNet("path/to/yolov3.weights", "path/to/yolov3.cfg")

# Define classes to detect
classes = []
with open("path/to/coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

# Set input size
input_size = 416

# Set minimum confidence threshold
confidence_threshold = 0.5

# Set non-maximum suppression threshold
nms_threshold = 0.4

# Open video stream
cap = cv2.VideoCapture("path/to/highway_video.mp4")

# Initialize variables
prev_frame_time = 0
new_frame_time = 0
prev_bbox = None
prev_time = None
speed = 0

while True:
    # Read a frame from the video stream
    ret, frame = cap.read()

    if ret:
        # Get frame dimensions
        height, width, _ = frame.shape

        # Create blob from frame
        blob = cv2.dnn.blobFromImage(frame, 1/255, (input_size, input_size), (0, 0, 0), swapRB=True, crop=False)

        # Pass blob through network
        net.setInput(blob)
        layer_names = net.getLayerNames()
        output_names = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
        outputs = net.forward(output_names)

        # Initialize lists for bounding boxes, confidences, and class IDs
        boxes = []
        confidences = []
        class_ids = []

        # Process outputs
        for output in outputs:
            for detection in output:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > confidence_threshold and classes[class_id] == 'car':
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)
                    w = int(detection[2] * width)
                    h = int(detection[3] * height)
                    x = int(center_x - w/2)
                    y = int(center_y - h/2)
                    boxes.append([x, y, w, h])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)

        # Apply non-maximum suppression
        indices = cv2.dnn.NMSBoxes(boxes, confidences, confidence_threshold, nms_threshold)

        # Calculate car speed
        if prev_bbox is not None and indices is not None:
            curr_bbox = None
            curr_time = None
            for i in indices:
                i = i[0]
                box = boxes[i]
                x, y, w, h = box
                if prev_bbox is not None and x <= prev_bbox[0] + prev_bbox[2] and x + w >= prev_bbox[0]:
                    curr_bbox = box
                    curr_time = time.time()
                    break
            if curr_bbox is not None and curr_time is not None and prev_time is not None:
                dt = curr_time - prev_time
                dx = curr_bbox[0] + curr_bbox[2]/2 - (prev_bbox[0] + prev_bbox[2]/2)
                speed = dx / dt * 3.6 # convert to km/h
            else:
                speed = 0
            prev_bbox = curr_bbox
            prev_time = curr_time

< cv2.VideoCapture 0x7f32fc29ffd0>
